In [138]:
from pprint import pprint
import boto3
import pandas as pd
from datetime import datetime, timedelta
from boto3.dynamodb.conditions import Key, Attr
import time

# GLOBAL VARIABLES

In [118]:
SAVE_CALCULATED_DF_PATH_BASE_FILENAME = 'sundaeswap_calculated_small.csv'
SCRAPPED_FOLDER_PATH = 'D:\ICARO\Proyectos\SeleniumSundaeSwap\Calculated'
BATCH_WRITE = True

# GET DATA

In [119]:
df = pd.read_csv(SCRAPPED_FOLDER_PATH+'\\'+SAVE_CALCULATED_DF_PATH_BASE_FILENAME, parse_dates=True)
df.head(3)

,PairPrice,TotalAssetLocked,TotalAdaLocked,AdaVolume24hs,DateTimeUnix,_PAIR_ID,_DELTA_PAIRPRICE,_DELTA_TOTALASSETLOCKED,_DELTA_TOTALADALOCKED,_DELTA_DATETIME_SECONDS
0,0.658510,21352876.90,14061097.46,1520497.36,1644722012,SUNDAE/ADA_0.3,NaN,NaN,NaN,NaN
1,85.410120,144162.15,12312906.74,635064.55,1644722012,LQ/ADA_0.3,NaN,NaN,NaN,NaN
2,0.785214,11970638.00,9399518.76,919539.05,1644722012,WMT/ADA_0.3,NaN,NaN,NaN,NaN


In [120]:
def unixToDatetime(ts: int):
    return datetime.utcfromtimestamp(ts)

dates = df['DateTimeUnix'].apply(unixToDatetime)
df['DateTime'] = dates

In [53]:
df['_PAIR_ID'].unique()

array(['SUNDAE/ADA_0.3', 'LQ/ADA_0.3', 'WMT/ADA_0.3', 'MELD/ADA_0.3',
       'PAVIA/ADA_0.3', 'XRAY/ADA_0.3', 'ADAX/ADA_0.3', 'AADA/ADA_0.3',
       'VYFI/ADA_0.3', 'MILK/ADA_0.3', 'MINt/ADA_0.3', 'CLAP/ADA_0.3',
       'CHRLZ/ADA_0.3', 'DANA/ADA_0.05', 'WMT/ADA_0.05', 'BISON/ADA_0.3',
       'ASHIB/ADA_1.0', 'FIRE/ADA_0.3', 'HOSKY/ADA_0.3', 'MIN/ADA_0.3',
       'cNETA/ADA_0.3', 'FLICK/ADA_0.3', 'CNFT/ADA_0.3', 'MELD/ADA_1.0',
       'REVU/ADA_0.3', 'BDOGE/ADA_1.0', 'MONET/ADA_0.3',
       'unsi…en/ADA_1.0', 'DRIP/ADA_1.0', 'MINt/ADA_0.05',
       'SKULL/ADA_1.0', 'RAG/ADA_1.0', 'MIN/ADA_0.05', 'GOKEY/ADA_0.3',
       'CDX/ADA_1.0', 'XDIA…ND/ADA_1.0', 'cMETA/ADA_0.3', 'RAVE/ADA_1.0',
       'DANA/ADA_0.3', 'CARGO/ADA_0.3', 'MEOW/ADA_0.3', 'SHIBA/ADA_1.0',
       'WMT/ADA_1.0', 'Card…rm/ADA_0.3', 'cbTHC/ADA_0.3',
       'HOSKY/ADA_0.05', 'DRIP/ADA_0.3', 'FLANO/ADA_1.0', 'KIDZ/ADA_0.3',
       'FIRE/ADA_1.0', 'YAY/ADA_1.0', 'QUID/ADA_0.3', 'SUNDAE/ADA_0.05',
       'LQ/ADA_1.0', 'SUNDAE

In [121]:
pairId = 'SUNDAE/ADA_0.3'
FILTER_DATE = datetime.fromisoformat('2022-02-13 12:00:00')
FILTER_DATE = datetime.fromisoformat('2022-02-15 12:00:00')
filter = (df['DateTime'] > FILTER_DATE) & (df['_PAIR_ID'] == pairId) #& (df['DateTimeUnix'] > 1645080041)

print(len(df[filter]))
row = df[df['_PAIR_ID']==pairId].tail(1)
row

198


,PairPrice,TotalAssetLocked,TotalAdaLocked,AdaVolume24hs,DateTimeUnix,_PAIR_ID,_DELTA_PAIRPRICE,_DELTA_TOTALASSETLOCKED,_DELTA_TOTALADALOCKED,_DELTA_DATETIME_SECONDS,DateTime
17480,0.760128,26180036.81,19900181.84,1156399.69,1645135241,SUNDAE/ADA_0.3,-0.000726,12550.62,-9475.97,900.0,2022-02-17 22:00:41


# TO DYNAMO DB

In [55]:
session = boto3.Session(region_name='us-east-1')
dynamodb = session.resource('dynamodb')

In [122]:
# table = dynamodb.Table('PythonTest')
table = dynamodb.Table('SundaeSwapPrices')

# response = table.put_item(Item=item)
# pprint(response, sort_dicts=False)

In [125]:
if BATCH_WRITE:
    pairIds = ['SUNDAE/ADA_0.3', 'LQ/ADA_0.3', 'WMT/ADA_0.3', 'MELD/ADA_0.3', 'MILK/ADA_0.3', 'MIN/ADA_0.3']
    print('BATCH START FOR PAIRS', len(pairIds))
    for pId in pairIds:
        filter = (df['DateTime'] > FILTER_DATE) & (df['_PAIR_ID'] == pId)
        print('BATCH UPDATE START', len(df[filter]))
        print('FOR',pId)
        with table.batch_writer() as batch:
            for index, row in df[filter].iterrows():
                # print(index,row['PairPrice'])
                # batch.put_item(
                #     Item={
                #     'Id': index,
                #     'IdSecond': pId+'_'+str(index),
                #     'PairId': pId,
                #     #'DateTimeUnix': str(row['DateTimeUnix']),
                #     'DateTimeUnix': row['DateTimeUnix'],
                #     'PairPrice': str(row['PairPrice']),
                #     '_DELTA_DATETIME_SECONDS': str(row['_DELTA_DATETIME_SECONDS']),
                #     '_DELTA_PAIRPRICE': str(row['_DELTA_PAIRPRICE'])
                # })
                batch.put_item(
                    Item={
                    'PairId': pId,
                    'DateTimeUnix': row['DateTimeUnix'],
                    'Index': index,
                    'PairPrice': str(row['PairPrice'])
                })
        print('BATCH UPDATE END (sleep), last DateTimeUnix', df[filter].DateTimeUnix.max())
        time.sleep(15)
    print('BATCH END')


BATCH START FOR PAIRS 6
BATCH UPDATE START 198
FOR SUNDAE/ADA_0.3
BATCH UPDATE END
BATCH UPDATE START 198
FOR LQ/ADA_0.3
BATCH UPDATE END
BATCH UPDATE START 198
FOR WMT/ADA_0.3
BATCH UPDATE END
BATCH UPDATE START 198
FOR MELD/ADA_0.3
BATCH UPDATE END
BATCH UPDATE START 198
FOR MILK/ADA_0.3
BATCH UPDATE END
BATCH UPDATE START 190
FOR MIN/ADA_0.3
BATCH UPDATE END
BATCH END


In [117]:
len(df[filter])

140

In [127]:
response = table.get_item(
    Key={
        'PairId': 'SUNDAE/ADA_0.3',
        'DateTimeUnix': 1645135241
    }
)
item = response['Item']
print(item)

{'PairId': 'SUNDAE/ADA_0.3', 'Index': Decimal('17480'), 'DateTimeUnix': Decimal('1645135241'), 'PairPrice': '0.760128'}


In [149]:
d = datetime.now() - timedelta(days=1, hours=0)
unixtime = time.mktime(d.timetuple())
ut = '{:.0f}'.format(unixtime)
print(ut)
print(type(ut))
print(type(int(ut)))
print(d)
print(datetime.now())

1645095093
<class 'str'>
<class 'int'>
2022-02-17 07:51:33.993874
2022-02-17 19:51:33.994890


In [140]:
response = table.scan(
    FilterExpression=Attr('PairId').eq('SUNDAE/ADA_0.3') & Attr('DateTimeUnix').gt(int(ut))
    #FilterExpression=Attr("Date").eq(date) & Attr("Shift").eq(shift)
)
items = response['Items']

In [141]:
print(len(items))
print(items)

0
[]


In [144]:
response = table.query(
        KeyConditionExpression=Key('PairId').eq('SUNDAE/ADA_0.3') & Key('DateTimeUnix').gt(int(ut))
    )
items = response['Items']

In [145]:
print(len(items))
print(items)

92
[{'PairId': 'SUNDAE/ADA_0.3', 'Index': Decimal('13210'), 'DateTimeUnix': Decimal('1645053367'), 'PairPrice': '0.744327'}, {'PairId': 'SUNDAE/ADA_0.3', 'Index': Decimal('13260'), 'DateTimeUnix': Decimal('1645054239'), 'PairPrice': '0.744327'}, {'PairId': 'SUNDAE/ADA_0.3', 'Index': Decimal('13310'), 'DateTimeUnix': Decimal('1645055135'), 'PairPrice': '0.744327'}, {'PairId': 'SUNDAE/ADA_0.3', 'Index': Decimal('13320'), 'DateTimeUnix': Decimal('1645056040'), 'PairPrice': '0.744327'}, {'PairId': 'SUNDAE/ADA_0.3', 'Index': Decimal('13370'), 'DateTimeUnix': Decimal('1645056942'), 'PairPrice': '0.744327'}, {'PairId': 'SUNDAE/ADA_0.3', 'Index': Decimal('13420'), 'DateTimeUnix': Decimal('1645057840'), 'PairPrice': '0.744327'}, {'PairId': 'SUNDAE/ADA_0.3', 'Index': Decimal('13470'), 'DateTimeUnix': Decimal('1645058741'), 'PairPrice': '0.744327'}, {'PairId': 'SUNDAE/ADA_0.3', 'Index': Decimal('13520'), 'DateTimeUnix': Decimal('1645059639'), 'PairPrice': '0.744327'}, {'PairId': 'SUNDAE/ADA_0.3',